In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-10"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9714,2024-05-10,Espanha Acb,14:00,Barcelona,Unicaja,1.65,2.26,165.5,1.85,1.93,-4.5,2.01,4.20,C2zptPcR,0.606061,0.442478,0.540541,0.518135,0.048538,0.4,0.6,NaN,NaN,128.010,28.272445,0.220861,141.978,32.537320,0.229172,111.614,144.062,147.05,163.40,0.0,0.0,0.0,0.0,0.220632,0.029930,0.498732,0.18,0.036,18.055556,0.000000,0.0,0.000000,0.00,0.000,inf,0.607899,0.8,0.192101
9715,2024-05-10,Espanha Acb,15:00,Breogan,MoraBanc Andorra,1.55,2.47,161.5,1.83,1.97,-4.5,1.90,3.75,Q9GKZMze,0.645161,0.404858,0.546448,0.507614,0.050020,0.6,0.8,NaN,NaN,220.152,133.944635,0.608419,210.746,178.133993,0.845254,206.324,329.654,109.48,123.90,0.0,0.0,0.0,0.0,0.323651,0.052103,0.463061,-3.47,-0.694,-0.792507,0.000000,0.0,0.000000,-5.00,-1.000,-1.470000,0.000000,0.0,0.000000
9716,2024-05-10,Espanha Acb,15:30,Zaragoza,Granada,1.47,2.71,168.5,1.87,1.89,-4.5,1.83,4.20,40FOYtj1,0.680272,0.369004,0.534759,0.529101,0.049276,0.6,0.4,NaN,NaN,170.334,55.651983,0.326723,215.314,101.911682,0.473317,159.108,305.768,179.10,194.74,0.0,0.0,0.0,0.0,0.419527,0.007522,0.555835,-0.13,-0.026,-18.076923,0.528433,0.7,0.171567,1.18,0.236,7.245763,0.000000,0.0,0.000000
9717,2024-05-10,Espanha Acb,16:00,Valencia,Real Madrid,2.38,1.59,166.5,1.87,1.91,2.5,2.01,1.31,bNESX057,0.420168,0.628931,0.534759,0.523560,0.049099,0.8,0.4,NaN,NaN,155.304,28.354995,0.182577,121.618,32.332730,0.265855,125.256,118.462,175.38,167.48,0.0,0.0,0.0,0.0,0.281418,0.014965,0.298178,-0.76,-0.152,-9.078947,0.706928,0.6,-0.106928,-2.24,-0.448,-1.316964,0.727660,0.7,-0.027660
9718,2024-05-10,Espanha Acb,16:30,Gran Canaria,Basquet Girona,1.13,6.10,168.5,1.86,1.92,-10.5,1.80,10.50,d8XVWKKD,0.884956,0.163934,0.537634,0.520833,0.048890,0.4,0.4,NaN,NaN,119.302,22.941766,0.192300,279.938,275.843295,0.985373,144.132,328.286,118.50,236.94,0.0,0.0,0.0,0.0,0.972150,0.022448,1.000297,3.01,0.602,0.215947,0.000000,0.0,0.000000,-5.00,-1.000,-5.100000,0.000000,0.0,0.000000
9719,2024-05-10,Eua Nba,20:00,Indiana Pacers,New York Knicks,1.36,3.31,222.5,1.93,1.93,-7.5,1.95,7.00,f1QxKVGl,0.735294,0.302115,0.518135,0.518135,0.037409,1.0,0.4,NaN,NaN,177.526,43.500639,0.245038,216.380,43.031112,0.198868,176.862,223.980,156.00,272.58,0.0,0.0,0.0,0.0,0.590517,0.000000,0.797964,2.27,0.454,0.792952,0.656890,0.8,0.143110,3.52,0.704,3.281250,0.522028,0.7,0.177972
9720,2024-05-10,Eua Nba,22:30,Minnesota Timberwolves,Denver Nuggets,1.55,2.56,204.5,1.93,1.93,-4.5,1.93,3.70,hKUzt3Fi,0.645161,0.390625,0.518135,0.518135,0.035786,0.8,0.6,NaN,NaN,246.812,61.452715,0.248986,160.056,37.676227,0.235394,206.316,180.058,174.30,173.88,0.0,0.0,0.0,0.0,0.347532,0.000000,0.444611,0.20,0.040,13.750000,0.633888,0.4,-0.233888,0.09,0.018,86.666667,0.569360,0.5,-0.069360
9721,2024-05-10,Alemanha Bbl,15:00,Alba Berlin,Bayern,2.17,1.70,160.5,1.86,1.92,1.5,2.00,1.55,lpoK2VhP,0.460829,0.588235,0.537634,0.520833,0.049065,1.0,0.4,NaN,NaN,137.822,28.995653,0.210385,122.962,15.013964,0.122102,116.898,131.114,114.48,132.03,0.0,0.0,0.0,0.0,0.171752,0.022448,0.179267,0.11,0.022,53.181818,0.000000,0.0,0.000000,2.12,0.424,1.650943,0.000000,0.0,0.000000
9722,2024-05-10,Alemanha Pro A,14:30,PS Karlsruhe,Giessen,1.83,1.89,168.5,1.87,1.79,-1.5,1.96,2.05,G2ZE310c,0.546448,0.529101,0.534759,0.558659,0.075549,0.6,0.8,NaN,NaN,163.348,38.298402,0.234459,138.770,25.560822,0.184196,127.680,155.826,171.50,162.36,0.0,0.0,0.0,0.0,0.022810,0.030912,0.031740,-0.65,-0.130,-6.384615,0.000000,0.0,0.000000,-2.21,-0.442

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,14:00,Espanha Acb,Barcelona,Unicaja,165.5,1.85,1.0000,Over
1,15:00,Espanha Acb,Breogan,MoraBanc Andorra,161.5,1.83,1.0000,Over
2,15:30,Espanha Acb,Zaragoza,Granada,168.5,1.87,1.0000,Over
3,16:30,Espanha Acb,Gran Canaria,Basquet Girona,168.5,1.86,1.0000,Over
4,20:00,Eua Nba,Indiana Pacers,New York Knicks,222.5,1.93,1.0000,Over
5,22:30,Eua Nba,Minnesota Timberwolves,Denver Nuggets,204.5,1.93,1.0000,Over
6,14:30,Alemanha Pro A,PS Karlsruhe,Giessen,168.5,1.87,1.0000,Over
7,15:00,Alemanha Pro A,Kirchheim,Phoenix Hagen,157.5,1.87,1.0000,Over
8,08:30,Filipinas Copa Das Filipinas,TNT Tropang Giga,Rain Or Shine Paint.,209.5,1.80,0.9721,Over
9,15:00,França Pro B,Denain-Voltaire,ASA,161.5,1.87,1.0000,Over
